In [3]:
# 01_build_player_snapshot.ipynb

import os
import numpy as np
import pandas as pd

# Notebook is inside /notebooks, data is in ../data
PROJECT_ROOT = os.path.dirname(os.getcwd())   # one level up from /notebooks
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

print("CWD:", os.getcwd())
print("DATA_DIR:", DATA_DIR)
print("FILES in data:", os.listdir(DATA_DIR))


CWD: c:\Users\Home\OneDrive\文件\GitHub\hackathon-2025-evan-ston-energy\data
DATA_DIR: c:\Users\Home\OneDrive\文件\GitHub\hackathon-2025-evan-ston-energy\data
FILES in data: ['appearances.csv', 'clubs.csv', 'club_games.csv', 'competitions.csv', 'data', 'dataprocess.ipynb', 'games.csv', 'game_events.csv', 'game_lineups.csv', 'players.csv', 'player_valuations.csv', 'processed', 'transfers.csv']


In [4]:
players = pd.read_csv(os.path.join(DATA_DIR, "players.csv"))
valuations = pd.read_csv(os.path.join(DATA_DIR, "player_valuations.csv"))
apps = pd.read_csv(os.path.join(DATA_DIR, "appearances.csv"))
games = pd.read_csv(os.path.join(DATA_DIR, "games.csv"))
clubs = pd.read_csv(os.path.join(DATA_DIR, "clubs.csv"))
comps = pd.read_csv(os.path.join(DATA_DIR, "competitions.csv"))
transfers = pd.read_csv(os.path.join(DATA_DIR, "transfers.csv"))
club_games = pd.read_csv(os.path.join(DATA_DIR, "club_games.csv"))

players.head(), valuations.head()


(   player_id first_name     last_name                name  last_season  \
 0         10   Miroslav         Klose      Miroslav Klose         2015   
 1         26      Roman  Weidenfeller  Roman Weidenfeller         2017   
 2         65    Dimitar      Berbatov    Dimitar Berbatov         2015   
 3         77        NaN         Lúcio               Lúcio         2012   
 4         80        Tom        Starke          Tom Starke         2017   
 
    current_club_id         player_code    country_of_birth city_of_birth  \
 0              398      miroslav-klose              Poland         Opole   
 1               16  roman-weidenfeller             Germany          Diez   
 2             1091    dimitar-berbatov            Bulgaria   Blagoevgrad   
 3              506               lucio              Brazil      Brasília   
 4               27          tom-starke  East Germany (GDR)       Freital   
 
   country_of_citizenship  ...   foot height_in_cm contract_expiration_date  \
 0   

In [5]:
snap = valuations.copy()
snap["snapshot_date"] = pd.to_datetime(snap["date"])
snap = snap.sort_values(["player_id", "snapshot_date"])

# 下一個估值（未來身價）
snap["future_snapshot_date"] = snap.groupby("player_id")["snapshot_date"].shift(-1)
snap["future_market_value"] = snap.groupby("player_id")["market_value_in_eur"].shift(-1)

# 距離未來估值的天數
snap["delta_days_to_future"] = (
    snap["future_snapshot_date"] - snap["snapshot_date"]
).dt.days

# 只保留「大約一季～一年之後」的 label（避免太近或太久）
valid_mask = snap["delta_days_to_future"].between(180, 540)
snap = snap[valid_mask].copy()

# log-growth label
snap["y_growth"] = np.log(snap["future_market_value"] + 1) - np.log(
    snap["market_value_in_eur"] + 1
)

snap[["player_id", "snapshot_date", "market_value_in_eur",
      "future_market_value", "delta_days_to_future", "y_growth"]].head()


,player_id,snapshot_date,market_value_in_eur,future_market_value,delta_days_to_future,y_growth
4755,10,2006-01-09,20000000,30000000.0,187.0,0.405465
6254,10,2006-07-15,30000000,23000000.0,341.0,-0.265703
9024,10,2007-06-21,23000000,20000000.0,349.0,-0.139762
16465,10,2008-06-04,20000000,18000000.0,371.0,-0.105361
38770,10,2010-05-08,7500000,7000000.0,249.0,-0.068993


In [6]:
p = players.copy()
p["date_of_birth"] = pd.to_datetime(p["date_of_birth"], errors="coerce")
p["contract_expiration_date"] = pd.to_datetime(
    p["contract_expiration_date"], errors="coerce"
)

base_cols = [
    "player_id",
    "name",
    "date_of_birth",
    "position",
    "sub_position",
    "foot",
    "height_in_cm",
    "country_of_citizenship",
    "current_club_id",
    "current_club_domestic_competition_id",
    "current_club_name",
    "market_value_in_eur",
    "highest_market_value_in_eur",
    "contract_expiration_date",
]
p = p[base_cols]

snap = snap.merge(p, on="player_id", how="left", suffixes=("", "_player"))

# 年齡
snap["age"] = (snap["snapshot_date"] - snap["date_of_birth"]).dt.days / 365.25

# 合約剩餘年數
snap["years_to_contract_end"] = (
    (snap["contract_expiration_date"] - snap["snapshot_date"]).dt.days / 365.25
)

# 巔峰身價比例
snap["mv_ratio_to_peak"] = snap["market_value_in_eur"] / snap[
    "highest_market_value_in_eur"
]

snap[["player_id", "snapshot_date", "name", "age", "position",
      "market_value_in_eur", "highest_market_value_in_eur",
      "mv_ratio_to_peak"]].head()


,player_id,snapshot_date,name,age,position,market_value_in_eur,highest_market_value_in_eur,mv_ratio_to_peak
0,10,2006-01-09,Miroslav Klose,27.586585,Attack,20000000,30000000.0,0.666667
1,10,2006-07-15,Miroslav Klose,28.098563,Attack,30000000,30000000.0,1.000000
2,10,2007-06-21,Miroslav Klose,29.032170,Attack,23000000,30000000.0,0.766667
3,10,2008-06-04,Miroslav Klose,29.987680,Attack,20000000,30000000.0,0.666667
4,10,2010-05-08,Miroslav Klose,31.912389,Attack,7500000,30000000.0,0.250000


In [7]:
# 把 season 加進 appearances
apps2 = apps.merge(games[["game_id", "season"]], on="game_id", how="left")

# 基本 aggregates：一季一個 row
perf_season = (
    apps2.groupby(["player_id", "season"])
    .agg(
        games_played=("appearance_id", "count"),
        minutes_total=("minutes_played", "sum"),
        goals_total=("goals", "sum"),
        assists_total=("assists", "sum"),
        yellow_cards_total=("yellow_cards", "sum"),
        red_cards_total=("red_cards", "sum"),
    )
    .reset_index()
)

# per-90
perf_season["goals_per_90"] = (
    perf_season["goals_total"] / perf_season["minutes_total"].replace(0, np.nan) * 90
)
perf_season["assists_per_90"] = (
    perf_season["assists_total"] / perf_season["minutes_total"].replace(0, np.nan) * 90
)
perf_season["minutes_per_game"] = (
    perf_season["minutes_total"] / perf_season["games_played"].replace(0, np.nan)
)

# 將 snapshot_date 映射到「賽季年份」：7 月之後算當季，之前算上一季
month = snap["snapshot_date"].dt.month
year = snap["snapshot_date"].dt.year
snap["season_year"] = np.where(month >= 7, year, year - 1)

# join 當季表現
snap = snap.merge(
    perf_season.rename(columns={"season": "season_year"}),
    on=["player_id", "season_year"],
    how="left",
)

# 計算前一季的表現 & delta
snap = snap.sort_values(["player_id", "season_year"])
for col in ["minutes_total", "goals_per_90", "assists_per_90", "minutes_per_game"]:
    snap[f"prev_{col}"] = snap.groupby("player_id")[col].shift(1)
    snap[f"delta_{col}"] = snap[col] - snap[f"prev_{col}"]

snap[
    [
        "player_id",
        "snapshot_date",
        "season_year",
        "minutes_total",
        "goals_per_90",
        "assists_per_90",
        "delta_minutes_total",
        "delta_goals_per_90",
        "delta_assists_per_90",
    ]
].head()


,player_id,snapshot_date,season_year,minutes_total,goals_per_90,assists_per_90,delta_minutes_total,delta_goals_per_90,delta_assists_per_90
0,10,2006-01-09,2005,NaN,NaN,NaN,NaN,NaN,NaN
1,10,2006-07-15,2006,NaN,NaN,NaN,NaN,NaN,NaN
2,10,2007-06-21,2006,NaN,NaN,NaN,NaN,NaN,NaN
3,10,2008-06-04,2007,NaN,NaN,NaN,NaN,NaN,NaN
4,10,2010-05-08,2009,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Club info
c = clubs[
    [
        "club_id",
        "name",
        "domestic_competition_id",
        "total_market_value",
        "squad_size",
        "average_age",
    ]
].rename(columns={"name": "club_name", "total_market_value": "club_total_market_value"})

# League info
comp = comps[
    [
        "competition_id",
        "competition_code",
        "name",
        "country_name",
        "is_major_national_league",
    ]
].rename(
    columns={
        "competition_id": "domestic_competition_id",
        "name": "league_name",
        "country_name": "league_country",
    }
)

# club -> league_strength (用該聯賽所有 club 市值平均作 proxy)
league_strength_df = (
    clubs.groupby("domestic_competition_id")["total_market_value"]
    .mean()
    .reset_index()
    .rename(columns={"total_market_value": "league_strength"})
)

comp = comp.merge(league_strength_df, on="domestic_competition_id", how="left")

# join club & league
snap = snap.merge(
    c, left_on="current_club_id", right_on="club_id", how="left", suffixes=("", "_club")
)
snap = snap.merge(
    comp,
    left_on="player_club_domestic_competition_id",
    right_on="domestic_competition_id",
    how="left",
)

snap["league_is_major"] = snap["is_major_national_league"].fillna(False).astype("int8")

# 簡單 top5 league flag（依國家）
top5_countries = {"England", "Spain", "Germany", "Italy", "France"}
snap["is_top5_league"] = snap["league_country"].isin(top5_countries).astype("int8")

snap[
    [
        "player_id",
        "snapshot_date",
        "current_club_name",
        "league_name",
        "league_country",
        "club_total_market_value",
        "league_strength",
        "league_is_major",
        "is_top5_league",
    ]
].head()


,player_id,snapshot_date,current_club_name,league_name,league_country,club_total_market_value,league_strength,league_is_major,is_top5_league
0,10,2006-01-09,Società Sportiva Lazio S.p.A.,serie-a,Italy,NaN,NaN,1,1
1,10,2006-07-15,Società Sportiva Lazio S.p.A.,serie-a,Italy,NaN,NaN,1,1
2,10,2007-06-21,Società Sportiva Lazio S.p.A.,serie-a,Italy,NaN,NaN,1,1
3,10,2008-06-04,Società Sportiva Lazio S.p.A.,serie-a,Italy,NaN,NaN,1,1
4,10,2010-05-08,Società Sportiva Lazio S.p.A.,serie-a,Italy,NaN,NaN,1,1


In [9]:
# club_games + games → 每季每隊的勝率與 goal diff
cg = club_games.merge(
    games[["game_id", "date", "season"]],
    on="game_id",
    how="left",
)

club_season_stats = (
    cg.groupby(["club_id", "season"])
    .agg(
        club_games_played=("game_id", "count"),
        club_wins=("is_win", "sum"),
        club_goals_for=("own_goals", "sum"),
        club_goals_against=("opponent_goals", "sum"),
    )
    .reset_index()
)

club_season_stats["club_win_rate"] = (
    club_season_stats["club_wins"] / club_season_stats["club_games_played"].replace(0, np.nan)
)
club_season_stats["club_goal_diff_per_game"] = (
    (club_season_stats["club_goals_for"] - club_season_stats["club_goals_against"])
    / club_season_stats["club_games_played"].replace(0, np.nan)
)

club_season_stats = club_season_stats.rename(columns={"season": "season_year"})

# join 到 snapshot（current_club_id + season_year）
snap = snap.merge(
    club_season_stats,
    left_on=["current_club_id", "season_year"],
    right_on=["club_id", "season_year"],
    how="left",
    suffixes=("", "_club_season"),
)

snap[
    [
        "player_id",
        "snapshot_date",
        "current_club_name",
        "club_win_rate",
        "club_goal_diff_per_game",
    ]
].head()


,player_id,snapshot_date,current_club_name,club_win_rate,club_goal_diff_per_game
0,10,2006-01-09,Società Sportiva Lazio S.p.A.,NaN,NaN
1,10,2006-07-15,Società Sportiva Lazio S.p.A.,NaN,NaN
2,10,2007-06-21,Società Sportiva Lazio S.p.A.,NaN,NaN
3,10,2008-06-04,Società Sportiva Lazio S.p.A.,NaN,NaN
4,10,2010-05-08,Società Sportiva Lazio S.p.A.,NaN,NaN


In [11]:
# === 轉會資訊 features ===

transfers2 = transfers.copy()
transfers2["transfer_date"] = pd.to_datetime(transfers2["transfer_date"], errors="coerce")

# 把 "23/24" -> 2023 這種 season_year（用前兩位 + 2000）
def season_str_to_year(s):
    if pd.isna(s):
        return np.nan
    first = int(str(s).split("/")[0])   # "23/24" -> 23
    return 2000 + first                 # -> 2023

transfers2["season_year"] = transfers2["transfer_season"].apply(season_str_to_year)

# 對每個 transfer 算是否 moved_to_bigger_club
from_club_mv = clubs[["club_id", "total_market_value"]].rename(
    columns={"club_id": "from_club_id", "total_market_value": "from_club_mv"}
)
to_club_mv = clubs[["club_id", "total_market_value"]].rename(
    columns={"club_id": "to_club_id", "total_market_value": "to_club_mv"}
)

transfers2 = transfers2.merge(from_club_mv, on="from_club_id", how="left")
transfers2 = transfers2.merge(to_club_mv, on="to_club_id", how="left")

transfers2["moved_to_bigger_club"] = (
    transfers2["to_club_mv"] > transfers2["from_club_mv"]
).astype("int8")

# player-season level summary (注意這裡改成用 season_year，而不是 transfer_season)
transfer_season = (
    transfers2.groupby(["player_id", "season_year"])
    .agg(
        has_recent_transfer=("transfer_date", "count"),
        any_moved_to_bigger=("moved_to_bigger_club", "max"),
    )
    .reset_index()
    .rename(
        columns={
            "has_recent_transfer": "has_recent_transfer_count",
            "any_moved_to_bigger": "moved_to_bigger_club_flag",
        }
    )
)

# 是否在該季有任何轉會
transfer_season["has_recent_transfer"] = (
    transfer_season["has_recent_transfer_count"] > 0
).astype("int8")

# merge 回 snapshot，用 ["player_id", "season_year"] 對齊
snap = snap.merge(
    transfer_season[["player_id", "season_year", "has_recent_transfer", "moved_to_bigger_club_flag"]],
    on=["player_id", "season_year"],
    how="left",
)

snap["has_recent_transfer"] = snap["has_recent_transfer"].fillna(0).astype("int8")
snap["moved_to_bigger_club_flag"] = snap["moved_to_bigger_club_flag"].fillna(0).astype("int8")

snap[
    [
        "player_id",
        "snapshot_date",
        "season_year",
        "has_recent_transfer",
        "moved_to_bigger_club_flag",
    ]
].head()


,player_id,snapshot_date,season_year,has_recent_transfer,moved_to_bigger_club_flag
0,10,2006-01-09,2005,0,0
1,10,2006-07-15,2006,0,0
2,10,2007-06-21,2006,0,0
3,10,2008-06-04,2007,0,0
4,10,2010-05-08,2009,0,0


In [12]:
# 可以在這裡丟掉太久以前的資料
snap = snap[snap["snapshot_date"].dt.year >= 2010].copy()

# 選一組核心欄位（你可以再微調）
core_cols = [
    # keys
    "player_id",
    "snapshot_date",
    "season_year",
    "name",  # player_name
    "age",
    "position",
    "sub_position",
    "foot",
    "height_in_cm",
    "country_of_citizenship",

    # market
    "market_value_in_eur",          # current MV
    "highest_market_value_in_eur",
    "mv_ratio_to_peak",
    "y_growth",
    "future_market_value",
    "years_to_contract_end",

    # performance level
    "minutes_total",
    "games_played",
    "minutes_per_game",
    "goals_per_90",
    "assists_per_90",

    # performance growth
    "delta_minutes_total",
    "delta_goals_per_90",
    "delta_assists_per_90",

    # club & league
    "current_club_id",
    "current_club_name",
    "club_total_market_value",
    "club_win_rate",
    "club_goal_diff_per_game",
    "league_name",
    "league_country",
    "league_strength",
    "league_is_major",
    "is_top5_league",

    # transfer
    "has_recent_transfer",
    "moved_to_bigger_club_flag",
]

# 確保真的存在於 dataframe
core_cols = [c for c in core_cols if c in snap.columns]
player_snapshot = snap[core_cols].copy()

player_snapshot.head(), player_snapshot.shape


(   player_id snapshot_date  season_year            name        age position  \
 4         10    2010-05-08         2009  Miroslav Klose  31.912389   Attack   
 5         10    2011-06-29         2010  Miroslav Klose  33.054073   Attack   
 6         10    2012-07-03         2012  Miroslav Klose  34.067077   Attack   
 7         10    2013-06-19         2012  Miroslav Klose  35.028063   Attack   
 8         10    2014-01-07         2013  Miroslav Klose  35.581109   Attack   
 
      sub_position   foot  height_in_cm country_of_citizenship  ...  \
 4  Centre-Forward  right         184.0                Germany  ...   
 5  Centre-Forward  right         184.0                Germany  ...   
 6  Centre-Forward  right         184.0                Germany  ...   
 7  Centre-Forward  right         184.0                Germany  ...   
 8  Centre-Forward  right         184.0                Germany  ...   
 
    club_total_market_value  club_win_rate  club_goal_diff_per_game  \
 4                 

In [13]:
out_path = os.path.join(PROCESSED_DIR, "player_snapshot.parquet")
player_snapshot.to_parquet(out_path, index=False)
print(f"Saved player_snapshot -> {out_path}  (rows={len(player_snapshot)})")


Saved player_snapshot -> c:\Users\Home\OneDrive\文件\GitHub\hackathon-2025-evan-ston-energy\data\processed\player_snapshot.parquet  (rows=169226)
